In [824]:
# -*- coding: utf8 -*-
from os.path import expanduser
home = expanduser("~")
repertoire=home+"/Copy/Cours/Bordeaux/L1-UE1/Kalaba-15/02-PseudoLatin"
serie=repertoire+"/"
variante=""
#########################IMPORTS############################################
import codecs, optparse
import re, random
import sys,os,time
import string
import yaml
import ParFuMor as PFM
from ParFuMor import *
import pickle

In [825]:
#########################VARIABLES##########################################
version=os.path.basename("__file__")
time_stamp='%s' % time.strftime("%y%m%d-%H%M")
debug=0
debug_now=1
print_no=False
print_taches=False
print_coffee=False
print_commands=True
print_phrases=True
print_ortho=True
print_phono=True
print_glose=False
print_lexique=False
print_cloze=False
print_racines=False
no_form="***"
# no_grapho=['dormir', 'lit']
# no_phono=['gros', 'coussin']
no_grapho=['petit','Nabil',"sur"]
no_phono=["grand","autruche",'dans']
phono_no=u"XXXXX"
grapho_no=u"XXXXX"

In [826]:
prononciationBegin=[
    "\\begin{center}",
        "\\begin{tabular}{lcc}",
        "\\toprule",
        "Graphie & Prononciation & Glose \\\\",
        "\\midrule"
        ]
prononciationEnd=[
        "\\bottomrule",
        "\\end{tabular}",
    "\\end{center}"
]

In [827]:
with open(serie+"Gloses.yaml", 'r') as stream:
    gloses=yaml.load(stream)
with open(serie+"Stems.yaml", 'r') as stream:
    stems=yaml.load(stream)
with open(serie+"Phonology.yaml", 'r') as stream:
    phonology=yaml.load(stream)
with open(serie+"MorphoSyntax.yaml", 'r') as stream:
    morphosyntax=yaml.load(stream)
with open(serie+"Tableaux.yaml", 'r') as stream:
    tableaux=yaml.load(stream)
with open(serie+"Hierarchie.pkl", 'rb') as input:
   PFM.hierarchieCF = pickle.load(input)
with open(serie+"Lexique.pkl", 'rb') as input:
   PFM.lexique = pickle.load(input)
with open(serie+"Regles.pkl", 'rb') as input:
   PFM.regles = pickle.load(input)


In [828]:
[stems["PREP"][x][0] for x in stems["PREP"]]

[u'\xe0', 'devant', 'avec', 'de', 'pour', 'sous', 'sur', 'dans']

####Définition des entêtes

In [829]:
#########################CONSTANTS##########################################
head = [
"\\begin{tabular}[t]{|l|l|l|}",
"\\addlinespace[-1.0em]\\hline",
"Mot & Roman & Glose  \\\\",
"\\hline\\strutgh{14pt}%"
]
head_n = [
"\\begin{tabular}[t]{|l|c|c|c|}",
"\\addlinespace[-1.0em]\\hline",
"Nom & Genre & C\\indice{1}C\\indice{2}C\\indice{3} & V\\indice{L}  \\\\",
"\\hline\\strutgh{14pt}%"
]
head_v = [
"\\begin{tabular}[t]{|l|c|c|}",
"\\addlinespace[-1.0em]\\hline",
"Verbe & Type & C\\indice{1}C\\indice{2}C\\indice{3} \\\\",
"\\hline\\strutgh{14pt}%"
]

In [830]:
tail = [
"\\hline"
"\\end{tabular}\\\\"
]

####Définition des structures pour impression

In [831]:
exemples=[]
accumulateur=[]
vocabulaire=[]
def accumulerMots(mot):
    accumulateur.append(mot)
    return
def ajouterExemple(exemple,printBool=False):
    if printBool:
        print exemple
    exemples.append(exemple.strip())
    del accumulateur[:]
    return
def ajouterVocabulaire(terme,printBool=False):
    if printBool:
        print terme
    vocabulaire.append(terme.strip())
    return

####Définition des segments

In [832]:
consonnes=phonology["consonnes"]
voyelles=phonology["voyelles"]
gabarits=phonology["gabarits"]
derives=phonology["derives"]
nom_classe=phonology["nom_classe"]
nom_apo=phonology["apophonies"]

nom_mut=phonology["mutations"]

####Définition des catégories

In [833]:
attributsDetAdjNom=["Genre","Nombre","Cas"]
valAttribut={}
for attribut in attributsDetAdjNom:
    if attribut in gloses["N"]:
        valAttribut[attribut]=gloses["N"][attribut]
    elif attribut in gloses["ADJ"]:
        valAttribut[attribut]=gloses["ADJ"][attribut]
    elif attribut in gloses["DET"]:
        valAttribut[attribut]=gloses["DET"][attribut]
    else:
        valAttribut[attribut]=[]
boolAttribut={}
for element in ["DET","ADJ","N"]:
    boolAttribut[element]={key:key in gloses[element] for key in attributsDetAdjNom}

#types=gloses["V"]["CF"]
casSyntagmes=morphosyntax["Cas"]
lexiquePrepositions=[stems["PREP"][x][0] for x in stems["PREP"]]
casPreposition={}
for preposition in lexiquePrepositions:
    prep=preposition.upper()
    if casSyntagmes and prep in casSyntagmes:
        casPreposition[preposition]=casSyntagmes[prep]
    elif casSyntagmes and "PREP" in casSyntagmes:
        casPreposition[preposition]=casSyntagmes["PREP"]
    else:
        casPreposition[preposition]=""
        
i=2
verbe_forme={}
for forme in morphosyntax["V"]["FormesBase"]:
    verbe_forme[i]=forme
    i+=1

In [834]:
#casSyntagmes

####Remplacement des numéros de personnes pour les noms de macro LaTeX

In [835]:
def remplacerPersonnes(chaine):
    chaine.replace("1","Un")
    return chaine

In [836]:
remplacerPersonnes("1")

'1'

In [837]:
def recoder(chaine,table):
    if type(chaine)==str:
        temp=unicode(chaine.decode('utf8')).translate(table)
        result=temp.encode('utf8')
    elif type(chaine)==unicode:
        result=chaine.translate(table)
    return result

accentedIn = unicode(phonology["translations"]["deaccent"]["in"])
deaccentIn = [ord(char) for char in accentedIn]
deaccentOut = unicode(phonology["translations"]["deaccent"]["out"])
deaccent = dict(zip(deaccentIn, deaccentOut))

In [838]:
syntagmes=morphosyntax["Syntagmes"]

In [839]:
contractions=morphosyntax["Contractions"]
for contraction in contractions:
    temp=[]
    for element in contractions[contraction]:
        if isinstance(element,unicode):
            temp.append(element.encode("utf8"))
        else:
            temp.append(element)
    contractions[contraction]=temp

In [840]:
syllabes=phonology["syllabes"]

In [841]:
def taches():
    seuil=14
    def makeStain():
        seed=random.randint(1,1000)
        x=random.gauss(10,5)-1
        y=random.gauss(2,1)
        minimum=random.gauss(.2,.1)+.1
        maximum=random.gauss(.1,.05)+.5
        return "\\taches{%s}{%s}{%s}{%s}{%s}"%(seed,x,y,minimum,maximum)

    if print_taches:
        n=random.gauss(10,2.5)
        if n<7:
            return ""
        elif n<=seuil:
            return makeStain()
        else:
            nTaches=int(n-seuil)
            stains=""
            for i in range(nTaches):
                stains+=makeStain()
            return stains
    else:
        return ""

In [842]:
def faire_tableau(tableau,tab=(head,tail,"")):
    if len(tableau)==0: return
    comment=tab[2]
    for element in tab[0]:
        ajouterVocabulaire(comment+element)
    for element in tableau:
        ajouterVocabulaire(comment+element)
    for element in tab[1]:
        ajouterVocabulaire(comment+element)

In [843]:
def print_tableaux(cols,tableau,texte="",debut=0,tab=(head,tail,"")):
    ajouterVocabulaire(tab[2]+"\\begin{multicols}{"+str(cols)+"}")
    if texte!="":
        (table,reste)=filtrer_tableau(tableau,texte)
    else:
        table=tableau
        reste=[]
    chunk=(len(table)-debut*cols)/cols+1
    faire_tableaux(table,debut,cols,tab)
    ajouterVocabulaire(tab[2]+"\\end{multicols}")
    echantillon=min(len(reste),3)
    if echantillon>0:
        for element in random.sample(reste,echantillon):
            colonnes=element.split("&")
            graphie=colonnes[0].strip()
            phonologie=colonnes[1].strip()
            glose=colonnes[2].strip()
            prononciationExtrait.append(graphie+"&")
            prononciationExtrait.append("\\blanc{%s}"%phonologie+"&")
            prononciationExtrait.append(glose)
            if debug: print "".join(prononciationExtrait)

In [844]:
def faire_tableaux(tableau,debut=16,nombre=1,tab=(head,tail,"")):
    reste=[]
    if debug: print nombre,debut,tableau
    if debut!=0:
        for i in range(nombre):
            faire_tableau(tableau[debut*i:debut*(i+1)],tab)
        table=tableau[debut*nombre:]
    else:
        table=tableau
    longueur=len(table)
    chunk=longueur/nombre+1
    if debug: print "CHUNKING : ",longueur, nombre, chunk, table
    if chunk<48:
        chunks=chunk
    else:
        chunks=48
        reste=table[48*nombre:]
    if debug: print "RESTE : ", chunk, reste
    for i in range(nombre):
        faire_tableau(table[chunks*i:chunks*(i+1)],tab)
    if reste:
        faire_tableaux(reste,0,nombre,tab)

In [845]:
def filtrer_tableau(tableau,filtre):
    presents=[]
    absents=[]
    for line in tableau:
        elements=line.split(" ")
        if elements[0] in filtre: 
            presents.append(line)
        else:
            absents.append(line)
    return (presents,absents)

In [846]:
def faire_gn(depart,cas):
    global erg_genre, erg_nombre, abs_genre, abs_nombre
    if debug: print "groupe depart :", depart
    groupe_nom=[]
    groupe_nom.append(depart[0])
    if debug: print depart[0]
    for mot in depart[1:]:
        if debug: print mot
        groupe_nom.extend(etendre_contraction([mot]))
    if debug: print "groupe nom :", groupe_nom
    mots=[]
    det=[]
    adj=[]
    nom=[]
    gp=[]
    structureSyntagme={key:[] for key in syntagmes['GN']}
    tete=""
    nombre=""
    reste=0
    for mot in groupe_nom:
        if reste==0:
#            if mot=="deux" and "Du" in nombresNom:
            if mot=="deux" and "Du" in valAttribut["Nombre"]:
                nombre="DU"
                if det==[]: det.append(PFM.lexique.formeLexeme["des"][0])
            else:
                nomLexeme=PFM.lexique.formeLexeme[mot][0]
                categorie=PFM.lexique.lexemes[nomLexeme].classe.split(".")[-1]
                if debug: 
                    print "mot",[mot]
                    print "vedette",PFM.lexique.formeLexeme[mot][0],categorie
                    print "categories",PFM.hierarchieCF.classes["N"],PFM.hierarchieCF.getCategory(categorie)
                if PFM.hierarchieCF.getCategory(categorie)=="N":
                    tete=categorie
                    if debug: print "tête :", tete
                    tampon=tete.split('.')
                    classe=tampon[0]
                    try:
                        typeMot=tampon[1]
                    except IndexError:
                        typeMot=''
                    if mot[len(mot)-1]=='s':
                        if nombre=="": nombre="PL"
                    else:
                        if nombre=="": nombre="SG"
                    nom.append(PFM.lexique.formeLexeme[mot][0])
#                    if typeCas=="NoCas":
#                        cas=""
                    if not boolAttribut["N"]["Cas"]:
                        cas=""
#                    else:
                    cellule=classe.capitalize()+nombre.capitalize()+cas.capitalize()
                    if cas=="ERG":
                        erg_genre=classe
                        erg_nombre=nombre
                        if debug: print "ERG",erg_genre, erg_nombre
                    elif cas=="ABS":
                        abs_genre=classe
                        abs_nombre=nombre
                        if debug: print "ABS",abs_genre, abs_nombre
                elif categorie in ["DET"]:
                    det.append(PFM.lexique.formeLexeme[mot][0])
                elif categorie in ["ADJ"] or ("ADJ" in PFM.hierarchieCF.classes and categorie in PFM.hierarchieCF.classes["ADJ"]):
                    adj.append(PFM.lexique.formeLexeme[mot][0])
                elif categorie=="PREP":			#Si on trouve une PREP, elle et le reste forment un GP
                    gp.append(mot)
                    reste=1
        else:							#On a trouvé une PREP, toute la suite va dans GP
            gp.append(mot)
    if debug: print "accord :", tete
    if reste==1: gp=faire_gp(gp)
    if debug: print "GP dans le GN : ", gp
    if debug: print "GN sans det ? ", det
    if not det: det.append("IND")
    tempSyntagme=[]
    for mot in det:
        if not boolAttribut["DET"]["Cas"]:
            cas=""
#		glose=faire_glose(mot,classe,type,nombre)
        ref="\\"+recoder(mot.split(".")[0],deaccent).upper()+classe.capitalize()+nombre.capitalize()+cas.capitalize()
#        mots.append(ref)
#        texte.append(ref)
        tempSyntagme.append(ref)
        texte.append(ref)
    structureSyntagme["DET"]=tempSyntagme
    mots.insert(syntagmes['GN'].index("DET")," ".join(structureSyntagme["DET"]))
    tempSyntagme=[]
    for mot in gp: 
#        mots.append(mot)
        tempSyntagme.append(mot)
    structureSyntagme["GP"]=tempSyntagme
    mots.insert(syntagmes['GN'].index("GP"),structureSyntagme["GP"])
    tempSyntagme=[]
    for mot in adj:
#		glose=faire_glose(mot,classe,type,nombre)
        if not boolAttribut["ADJ"]["Cas"]:
            cas=""
        ref="\\"+recoder(mot.split(".")[0],deaccent).lower()+classe.capitalize()+nombre.capitalize()+cas.capitalize()
#        mots.append(ref)
        texte.append(ref)
        tempSyntagme.append(ref)
    structureSyntagme["GADJ"]=tempSyntagme
    mots.insert(syntagmes['GN'].index("GADJ"),structureSyntagme["GADJ"])
    tempSyntagme=[]
    for mot in nom:
#		glose=faire_glose(mot,classe,type,nombre)
        if mot.istitle():
            ref="\\"+recoder(mot.split(".")[0],deaccent)+cellule
        else:
            ref="\\"+recoder(mot.split(".")[0],deaccent).lower()+cellule
#        mots.append(ref)
        texte.append(ref)
        tempSyntagme.append(ref)
    structureSyntagme["N"]=tempSyntagme
    mots.insert(syntagmes['GN'].index("N"),structureSyntagme["N"])
    listeMots=[]
    for element in syntagmes['GN']:
        listeMots+=structureSyntagme[element]
    return listeMots

In [847]:
def faire_gp(groupe_prep,fonction=""):
    mots=[]
    groupe_prep=etendre_contraction(groupe_prep)
    if debug: print "faire_gp", groupe_prep
    preposition=groupe_prep[0]
#    if preposition!="à" or typeCas=="NoCas":
    if not boolAttribut["N"]["Cas"] or fonction!="IND":
        if debug: print u"fonction!=IND",[groupe_prep[0],u"à"]
        if casPreposition and preposition in casPreposition:
            cas=casPreposition[preposition]
            ref="\\"+recoder(groupe_prep[0],deaccent).upper()
            if "+" in casPreposition[preposition]:
                cas=cas.strip("+")
                mots.append(ref)
                texte.append(ref)
        else:
            cas=""
        if debug:
            print "groupe prep :", groupe_prep
            print ref
        if len(groupe_prep)>1:
            groupe_nom=groupe_prep[1:]
            #
            # Choisir le cas en fonction de la préposition
            #
            mots.insert(syntagmes['GP'].index("GN"),faire_gn(groupe_nom,cas))
        if debug: 
            print groupe_prep, cas, mots
        return mots
    elif fonction=="IND":
        groupe_nom=groupe_prep[1:]
        cas=casSyntagmes["IND"]
        if debug: print "faire_gn", groupe_nom, faire_gn(groupe_nom,cas)
        mots.append(faire_gn(groupe_nom,cas))
        return mots

In [848]:
def etendre_contraction(liste):
    result=[]
    if liste[0] in contractions.keys():
        if debug: print "EXT : ", liste, contractions[liste[0]],liste[1:] 
        result.extend(contractions[liste[0]])
        result.extend(liste[1:])
    else:
        result=liste
    return result

In [849]:
def printflat(liste,suffixe="",prefixe=""):
    if debug: print "printflat", liste
    if not isinstance(liste, basestring):
        for element in liste:
            accumulerMots(prefixe)
            printflat(element,suffixe)
    else: 
        accumulerMots(prefixe)
        accumulerMots(liste+suffixe)

In [850]:
#######################
#
#	INITIALISATION DES VARIABLES
#
#######################

try:
    __IPYTHON__ 
    ipython=True
except: 
    ipython=False

In [851]:
################
#
# LECTURE DU FICHIER DE LEXEMES
#
#		LES LIGNES QUI COMMENCENT PAR # SONT IGNOREES
#
################
print "%% version : "+version
print "%% traitement : "+time_stamp

if ipython or True:
#    lexeme_nom=serie+"Lexemes.txt"
#    phrase_nom=serie+"Phrases.txt"
    phrase_nom=serie+variante+"Phrases.csv"
else:
    parser=optparse.OptionParser()
    parser.add_option("-o", "--out", dest="outfile", action="store_true", help="write to FILE")
    parser.add_option("-c", "--cloze", dest="print_cloze", action="store_true", help="write a CLOZE FILE")
    parser.add_option("-l", "--lexicon", dest="print_lexique", action="store_true", help="append a lexicon")
    parser.add_option("-r", "--roots", dest="print_racines", action="store_true", help="append a root list")

    (options, args) = parser.parse_args()
    lexeme_nom=args[0]
    phrase_nom=args[1]

%% version : __file__
%% traitement : 150922-1504


####Ouverture du fichier lexique

####Ouverture du fichier phrases

In [852]:
try:
    phrase_file = codecs.open(phrase_nom,"r","utf8")
except IOError:
    print 'I could not open the sentence file', phrase_nom
    sys.exit()

In [853]:
def recoder(chaine,table):
    if type(chaine)==str:
        temp=unicode(chaine.decode('utf8')).translate(table)
        result=temp.encode('utf8')
    elif type(chaine)==unicode:
        result=chaine.translate(table)
    return result

In [854]:
accentedIn = unicode(phonology["translations"]["deaccent"]["in"])
deaccentIn = [ord(char) for char in accentedIn]
deaccentOut = unicode(phonology["translations"]["deaccent"]["out"])
deaccent = dict(zip(deaccentIn, deaccentOut))

In [855]:
tipaIn = unicode(phonology["translations"]["ipa"]["in"])
ipaIn = [ord(char) for char in tipaIn]
ipaOut = unicode(phonology["translations"]["ipa"]["out"])
toipa = dict(zip(ipaIn, ipaOut))

In [856]:
#################################################
#################################################
#################################################
##
##
##	FAIRE LE TRI DES FORMES UTILISEES DANS LES PHRASES
##	AFFICHER DANS LES TABLEAUX SEULEMENT CES FORMES
##
##
#################################################
################################################
#
#
#	FAIRE LA LISTE DES PHRASES AVEC LES 4 LIGNES
#		GRAPHO, PHONO, GLOSE, TRAD
#
#
################################################
texte=[]
#graphies={}
#abs_genre=""
#abs_nombre=""

In [857]:
if print_phrases:
    comment=""
else:
    comment="%"
ajouterExemple(comment+"\\begin{exe}")
for line in phrase_file:
    phrase=[0 for i in range(len(syntagmes['Phrase']))]
    tampon=(line.strip().rstrip('.')).replace("'"," ").split("\t")
#    print tampon[0],type(tampon[0])
    if not tampon[0].startswith("#"):
        verbe=tampon[1].split(" ")
        verbeForme=verbe[0]
        if len(PFM.lexique.formeLexeme[verbeForme])!=1:
            print "FORME AMBIGUË"
        verbeLexeme=PFM.lexique.formeLexeme[verbeForme][0]
        temp=verbeLexeme.split(".")
        formeCitation=temp[0]
        typeVerbe=temp[1]
#        print verbeLexeme, formeCitation,typeVerbe
        verbeLemme="%s%s"%(formeCitation,typeVerbe.capitalize())
        verbeFormeIndex=PFM.lexique.lexemes[verbeLexeme].formes.index(verbeForme)
        if debug: print "verbe :", verbe
#        if verbeLexeme.endswith("VI"):
        if casSyntagmes and "SUJ" in casSyntagmes and "Erg" in casSyntagmes["SUJ"]:
            if ".VI" in verbeLexeme:            
                suj_cas="Abs"
            else:
                suj_cas="Erg"
                obj_cas="Erg"
        elif casSyntagmes and "SUJ" in casSyntagmes and "Nom" in casSyntagmes["SUJ"]:
            suj_cas=casSyntagmes["SUJ"]
            obj_cas=casSyntagmes["OBJ"]
        else:
            suj_cas=""
            obj_cas=""
        suj_genre=valAttribut["Genre"][0]
        suj_nombre=valAttribut["Nombre"][0]
        obj_genre=valAttribut["Genre"][0]
        obj_nombre=valAttribut["Nombre"][0]
        abs_genre=valAttribut["Genre"][0]
        abs_nombre=valAttribut["Nombre"][0]
        sujet=tampon[0].strip().split(" ")
        phrase[syntagmes['Phrase'].index('SUJ')]=faire_gn(sujet,suj_cas)
        if debug: print "sujet :",phrase[1]
        if len(tampon)>=3:
            objet=tampon[2].split(" ")
            if debug: print "objet : ",objet
            if objet!=['']: phrase[syntagmes['Phrase'].index('OBJ')]=faire_gn(objet,obj_cas)
        if len(tampon)>=4:
            indirect=tampon[3].split(" ")
            if debug: print "indirect : ",indirect
            if indirect!=['']: phrase[syntagmes['Phrase'].index('IND')]=faire_gp(indirect,"IND")
        if len(tampon)>=5:
            comp=tampon[4].split(" ")
            if comp!=['']:
                phrase[syntagmes['Phrase'].index('COMP')]=faire_gp(comp)
        if len(tampon)>=6:
            ajout=tampon[5].split(" ")
            phrase[syntagmes['Phrase'].index('AJOUT')]=faire_gp(ajout)
#        if typeCas=="NoCas":
        if not boolAttribut["N"]["Cas"] or suj_cas=="Nom":
            glose="\\"+recoder(formeCitation,deaccent)+morphosyntax["V"]["FormesBase"][verbeFormeIndex].capitalize()+morphosyntax["V"]["FormesPers"][verbeFormeIndex] 
        else:
            glose="\\"+recoder(formeCitation,deaccent)+morphosyntax["V"]["FormesBase"][verbeFormeIndex].capitalize()+abs_genre.capitalize()+"Trois"+abs_nombre.capitalize()
        phrase[syntagmes['Phrase'].index('V')]=glose
        texte.append(glose)
        if print_glose:
            ajouterExemple(comment+"\\ex\\glll")
        else:
            ajouterExemple(comment+"\\ex\\gll")
        print comment,
        for mot in phrase:
            if mot!=0:
                printflat(mot,"{}")
        ajouterExemple(" ".join(accumulateur)+"\\\\")
        print comment,
        for mot in phrase:
            if mot!=0:
                printflat(mot,"P{}")
        ajouterExemple(" ".join(accumulateur)+"\\\\")
        if print_glose:
            print comment,
            for mot in phrase:
                if mot!=0:
                    printflat(mot,"G{}")
            ajouterExemple(" ".join(accumulateur)+"\\\\")
        traduction=(line.strip().rstrip('.')).split()
        start=1
#        accumulerMots(comment)
        for element in traduction:			# convertir les S majuscules à la finale des mots en minuscules
            if element!="":
                if start:
                    start=0
                    element=element.capitalize()
                caracteres=list(element)
                if caracteres[len(caracteres)-1]=='S':
                    caracteres[len(caracteres)-1]='s'
                accumulerMots("".join(caracteres).encode('utf8'))
        ajouterExemple(taches()+" ".join(accumulateur))
        del accumulateur[:]
        if print_coffee and random.randint(1,4)==1:
            stain=random.choice(["A","B","C","D"])
            alpha=random.random()/1.5
            angle=random.randint(0,360)
            xoff=random.randint(-200,0)
            ajouterExemple('\\\\\\cofe%sm{%.3f}{1}{%d}{%d}{0}' % (stain,alpha,angle,xoff))
    if comment=="%": ajouterExemple("\\\\")
ajouterExemple(comment+"\\end{exe}")
    
phrase_file.close()

In [858]:
#PFM.hierarchieCF.classes

In [859]:
if ('options' in globals() and options.print_cloze) or print_lexique:
    tab=(head,tail,"")
else:
    tab=(head,tail,"%")
prononciationExtrait=[]
ajouterVocabulaire(tab[2]+"\\begin{itemize}")
ajouterVocabulaire(tab[2]+"\\item NOMS\\\\[-3ex]")
print_tableaux(2,tableaux["N"],texte,8,tab)
ajouterVocabulaire(tab[2]+"\\item ADJECTIFS\\\\[-3ex]")
print_tableaux(3,tableaux["ADJ"],texte,11,tab)
ajouterVocabulaire(tab[2]+"\\item VERBES\\\\[-3ex]")
print_tableaux(2,tableaux["V"],texte,35,tab)
ajouterVocabulaire(tab[2]+"\\item DÉTERMINANTS\\\\[-3ex]")
print_tableaux(3,tableaux["DET"],texte,12,tab)
ajouterVocabulaire(tab[2]+"\\item PRÉPOSITIONS\\\\[-3ex]")
print_tableaux(3,tableaux["PREP"],texte,0,tab)
ajouterVocabulaire(tab[2]+"\\end{itemize}")

if ('options' in globals() and options.print_cloze) or print_cloze:
    print
    print
    print
    print "%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%CLOZE%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%"
    print
    print
    print
    print "%										NOMS"	
    for element in cloze_noms:
        print "% ", element[0].translate(toipa)+";"+element[1]
    print
    print
    print
    print "%										VERBES"	
    for element in cloze_verbes:
        print "% ", element[0].translate(toipa)+";"+element[1]		

In [860]:
with open(serie+variante+"Exemples.tex", 'wb') as output:
    for exemple in exemples:
        output.write(exemple+"\n")

In [861]:
with open(serie+variante+"Vocabulaire.tex", 'wb') as output:
    for vocable in vocabulaire:
        output.write(vocable+"\n")

In [862]:
with open(serie+variante+"Prononciation.tex", 'wb') as output:
    for ligne in prononciationBegin+prononciationExtrait+prononciationEnd:
        output.write(ligne+"\n")
